In [1]:
# suppress qiskit 1.0 deprecation warnings:
import warnings
import numpy as np

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

from qiskit import QuantumCircuit, Aer, execute
import compile_keys as keys
import qcm_sim as sim

In [2]:
# import circuits:
import circuits.hidden_shift as hshift 
import circuits.ccz_7T_decomposition as ccz

m = 3
n = 2*m

# Generate a random hidden shift
shift = [np.random.choice(2) for _ in range(n)]

qc = hshift.hidden_shift(3,shift,is_clifford = False)

print(qc)

     ┌───┐          ┌───┐┌───┐               ┌───┐          ┌───┐     ┌─┐     »
q_0: ┤ H ├───────■──┤ S ├┤ S ├──■─────────■──┤ H ├───────■──┤ H ├─────┤M├─────»
     ├───┤       │  └───┘├───┤┌─┴─┐┌───┐  │  ├───┤┌───┐  │  └───┘┌───┐└╥┘     »
q_1: ┤ H ├───────┼────■──┤ H ├┤ X ├┤ H ├──■──┤ S ├┤ S ├──┼────■──┤ H ├─╫──────»
     ├───┤       │    │  └───┘└───┘└───┘  │  ├───┤└───┘  │  ┌─┴─┐├───┤ ║ ┌───┐»
q_2: ┤ H ├───────┼────┼────■──────────────■──┤ H ├───────┼──┤ X ├┤ H ├─╫─┤ H ├»
     ├───┤┌───┐┌─┴─┐  │    │  ┌───┐┌───┐┌───┐└───┘     ┌─┴─┐├───┤├───┤ ║ ├───┤»
q_3: ┤ H ├┤ H ├┤ X ├──┼────┼──┤ H ├┤ H ├┤ H ├──────────┤ X ├┤ H ├┤ S ├─╫─┤ S ├»
     ├───┤├───┤└───┘┌─┴─┐  │  ├───┤├───┤├───┤          └───┘└───┘└───┘ ║ └───┘»
q_4: ┤ H ├┤ H ├─────┤ X ├──┼──┤ H ├┤ H ├┤ H ├──────────────────────────╫──────»
     ├───┤├───┤     └───┘┌─┴─┐├───┤├───┤├───┤                          ║      »
q_5: ┤ H ├┤ H ├──────────┤ X ├┤ H ├┤ H ├┤ H ├──────────────────────────╫──────»
     └───┘└───┘          └───┘└───┘└───┘

In [3]:
# Qiskit simulation
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)
result = job.result()
counts = result.get_counts()

counts

{'001000': 1024}

In [4]:
# Decompose ccz gates as t gates
qc_w_ccz_decomposition = ccz.apply_ccz_via_7t_decomposition(qc)
qc_w_ccz_decomposition_qasm = qc_w_ccz_decomposition.qasm()

# number of samples:
negativity = keys.compute_total_negativity(14)
epsilon = 0.1
prob_fail = 0.01
hoeffding_samples = (negativity**2) * (2 / (epsilon**2)) * np.log(2 / prob_fail)

In [5]:
# Shift string
print(f"\nHidden Shift: {shift}\n")

# QCM simulation
results = sim.run_qcm(qc_w_ccz_decomposition_qasm,shots=hoeffding_samples/200)


Hidden Shift: [0, 0, 0, 1, 0, 0]

------------------------------
Initial conditions for simulation:
------------------------------

Negativity:
 31.103388466421812

Number of shots:
 5125

------------------------------
Outputs of Simulation:
------------------------------

Average time per shot: 0.002211 seconds

Distinct Outcome: (0, 0, 0, 0, 1, 1)
Born Rule Estimate: 0.09102183481785824

Distinct Outcome: (0, 0, 0, 1, 0, 1)
Born Rule Estimate: -0.04247685624833384

Distinct Outcome: (0, 0, 0, 0, 0, 0)
Born Rule Estimate: -0.26699738213238416

Distinct Outcome: (0, 0, 0, 1, 0, 0)
Born Rule Estimate: 1.1226026294202516

Distinct Outcome: (0, 0, 0, 0, 1, 0)
Born Rule Estimate: 0.12136244642381099

Distinct Outcome: (0, 0, 0, 1, 1, 1)
Born Rule Estimate: -0.15777118035095428

Distinct Outcome: (0, 0, 0, 0, 0, 1)
Born Rule Estimate: 0.07281746785428658

Distinct Outcome: (0, 0, 0, 1, 1, 0)
Born Rule Estimate: -0.012136244642381097

